##Pipeline


In [ ]:
import pandas as pd
import numpy as np
import spacy
import re
from scipy.spatial import distance
import math
import pickle
import string
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
import string
nltk.download('punkt')
nltk.download('stopwords')
# from readability import Readability
#

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#********Lexical Features********
# syllable count

def syllable_count(word):
    word = word.lower()
    count = 0
    vowels = "aeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
            if word[index]=='e':
                count -= 1
    if count == 0:
        count += 1
    return count

def avg_syllable_count(text):
    tokens = sent_tokenize(text)
    l1 = [word for token in tokens for word in token.split()]
    print(tokens)
    print(l1)
    # print([syllable_count(word) for word in l1])
    return np.average([syllable_count(word) for word in l1])


#avg word length

def avg_wordlength(text):
    string1 = text.translate(string.punctuation)
    # print(string1)
    tokens = word_tokenize(string1, language='english')
    # print(tokens)
    st = [",", ".", "'", "!", '"', "#", "$", "%", "&", "(", ")", "*", "+", "-", ".", "/", ":", ";", "<", "=", '>', "?",
          "@", "[", "\\", "]", "^", "_", '`', "{", "|", "}", '~', '\t', '\n']
    stop = stopwords.words('english') + st
    words = [word for word in tokens if word not in stop]
    return np.average([len(word) for word in words])


#avg sentence length by number of words

def avg_sentlengthbyword(text):
    tokens = sent_tokenize(text)
    return np.average([len(token.split()) for token in tokens])


def count_functionalwords(text):
    functional_words = """a between in nor some upon
    about both including nothing somebody us
    above but inside of someone used
    after by into off something via
    all can is on such we
    although cos it once than what
    am do its one that whatever
    among down latter onto the when
    an each less opposite their where
    and either like or them whether
    another enough little our these which
    any every lots outside they while
    anybody everybody many over this who
    anyone everyone me own those whoever
    anything everything more past though whom
    are few most per through whose
    around following much plenty till will
    as for must plus to with
    at from my regarding toward within
    be have near same towards without
    because he need several under worth
    before her neither she unless would
    behind him no should unlike yes
    below i nobody since until you
    beside if none so up your
    """

    functional_words = functional_words.split()
    tokens = sent_tokenize(text)
    wo_punc = [text.translate(str.maketrans ('', '', string.punctuation)) for text in tokens]
    l1 = [word for token in wo_punc for word in token.split()]
    count = 0

    for i in l1:
        if i in functional_words:
            count += 1

    return count / len(l1)


#punc count

def punctuation_count_ratio(text):
    st = [",", ".", "'", "!", '"', ";", "?", ":", ";"]
    count = 0
    for i in text:
        if (i in st):
            count = count + 1
    text = text.replace(" ", "")
    # print(text)
    return float(count) / float(len(text))


#********Readability score********
!pip install readability

import readability
#https://github.com/burrsettles/readability/blob/master/readability.py


#*******Semantic spread*******

!pip install spacy
!python -m spacy download en_core_web_md
!python -m spacy download nl_core_news_lg
!pip install spacy-transformers
nlp = spacy.load("en_core_web_md")
# nlp_trf = spacy.load("en_core_web_trf")
nlp_nl = spacy.load("nl_core_news_lg")


def remove_newlines(text):
    cleaned_text = re.sub(r'\n', '', text)
    return cleaned_text


def calculate_semantic_spread(paragraph, lang):
    doc = nlp(paragraph) if lang=="en" else nlp_nl(paragraph)
    sentences = [sent.text for sent in doc.sents]
    # print(sentences)
    spreads = []
    for sentence in sentences:
        sentence_doc = nlp(sentence) if lang=="en" else nlp_nl(sentence)
        # print(sentence_doc)
        vectors = [token.vector for token in sentence_doc]
        # print(vectors)
        similarities = distance.pdist(vectors, metric="cosine")
        # print(similarities)
        spread = np.nanstd(similarities)
        spreads.append(spread)
    return np.nanmean(spreads)

def pos_calculate_semantic_spread(text, lang):
  doc = nlp(text) if lang=="en" else nlp_nl(text)
  sentences = [sent.text for sent in doc.sents]
  # print(sentences)
  len_tokens = len([tok.text for tok in doc if not tok.is_punct])

  spreads = []
  noun_count = 0
  verb_count = 0
  adj_count = 0
  adv_count = 0
  # print(len_tokens)
  if len_tokens <= 1:
        return 0.0

  for sentence in sentences:
    sentence_doc = nlp(sentence) if lang=="en" else nlp_nl(sentence)
    # doc = nlp(text) if lang=="en" else nlp_nl(text)
    for token in sentence_doc:
        if token.pos_ == "NOUN":
            noun_count += 1
        elif token.pos_ == "VERB":
            verb_count += 1
        elif token.pos_ == "ADJ":
            adj_count += 1
        elif token.pos_ == "ADV":
            adv_count += 1
        # print(token, token.similarity(doc))
        # total_similarity += token.similarity(doc)

  # print(noun_count, verb_count, adj_count, adv_count)
  noun_ratio = noun_count / len_tokens
  verb_ratio = verb_count / len_tokens
  adj_ratio = adj_count / len_tokens
  adv_ratio = adv_count / len_tokens

  semantic_spread = (noun_ratio + verb_ratio + adj_ratio + adv_ratio) / 4
  # average_similarity = total_similarity / token_count

  return semantic_spread


#*******Lexical diversity*********
!pip install lexicalrichness
from lexicalrichness import LexicalRichness

In [ ]:
def style_score(text):
  #lexical features
  avg_syllable_count0 = avg_syllable_count(text)
  avg_wordlength0 = avg_wordlength(text)
  avg_sentlengthbyword0 = avg_sentlengthbyword(text)
  count_functionalwords0 = count_functionalwords(text)
  punctuation_count_ratio0 = punctuation_count_ratio(text)

  #readability scores
  results = readability.getmeasures(text, lang='en')
  fleschreadingease = results['readability grades']['FleschReadingEase']
  kincaid = results['readability grades']['Kincaid']
  gunningfogindex = results['readability grades']['GunningFogIndex']
  dalechallindex = results['readability grades']['DaleChallIndex']

  #Vocab richness
  lex = LexicalRichness(text)
  yulek = lex.yulek
  herdan = lex.Herdan
  maas = lex.Maas
  msttr = lex.msttr(segment_window=15)
  mattr = lex.mattr(window_size=15)
  simpsond = lex.simpsond

  #Semantic spread
  sem_spread = calculate_semantic_spread(text, 'en')
  pos_sem_spread = pos_calculate_semantic_spread(text, 'en')

  vector = [avg_syllable_count0, avg_wordlength0, avg_sentlengthbyword0, count_functionalwords0, punctuation_count_ratio0,

            fleschreadingease, kincaid, gunningfogindex, dalechallindex,

            yulek, herdan, maas, msttr, mattr, simpsond,

            sem_spread, pos_sem_spread
            ]

  style_dict = {}
  count = 0
  for metric in ['avg_syllable_count', 'avg_wordlength', 'avg_sentlengthbyword', 'count_functionalwords', 'punctuation_count_ratio', 'fleschreadingease', 'kincaid', 'gunningfogindex', 'dalechallindex', 'yulek', 'herdan', 'maas', 'msttr', 'mattr', 'simpsond','sem_spread', 'pos_sem_spread']:
    style_dict[metric] = vector[count]
    count+=1

  return vector, style_dict

##Data formation

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/aiGenTxt/pos_tagged.csv")

In [ ]:
df = df[:3000]

In [ ]:
count=-1
length = []
for text in df['text']:
  count+=1
  length.append((count,len(word_tokenize(text))))
  print(count, len(word_tokenize(text)))


In [ ]:
length.sort(key=lambda a:a[1])

In [ ]:
print(length)

[(1691, 15), (1337, 16), (1438, 19), (2318, 21), (737, 22), (1302, 23), (2656, 24), (1761, 25), (144, 26), (751, 26), (1934, 26), (756, 29), (2205, 30), (1899, 32), (2996, 32), (368, 33), (843, 33), (1196, 33), (2482, 36), (2701, 36), (2803, 37), (340, 38), (1182, 38), (1719, 38), (2191, 38), (41, 39), (587, 39), (669, 39), (2341, 40), (485, 41), (888, 41), (966, 41), (2704, 41), (1242, 42), (2004, 44), (2238, 44), (2908, 44), (1104, 45), (1773, 46), (1809, 47), (1977, 47), (430, 48), (555, 48), (2121, 48), (393, 49), (1080, 49), (1555, 49), (1965, 49), (2084, 49), (2510, 49), (2693, 49), (334, 50), (372, 51), (537, 51), (2286, 51), (2523, 51), (2778, 51), (113, 52), (261, 52), (1655, 52), (2204, 52), (2314, 52), (2565, 52), (2766, 52), (1635, 53), (1126, 54), (1447, 54), (200, 55), (1904, 55), (2665, 55), (1038, 56), (1412, 56), (2885, 56), (296, 57), (948, 57), (1097, 57), (1663, 57), (2106, 57), (345, 58), (1036, 58), (1587, 58), (2686, 58), (1000, 59), (2094, 59), (2547, 59), (2633

In [ ]:
df.drop([1691,1337,1438],axis=0,inplace=True)

In [ ]:
for x, y in enumerate(length):
    if y[1] == 19 and y[0]<800:
      print(y)

In [ ]:
vectors = []
count=-1
for text in df['text']:
  count+=1
  print(count)
  vectors.append(style_score(text))

In [ ]:
print(vectors[0])

([1.2864864864864864, 6.848484848484849, 23.125, 0.42162162162162165, 0.021253985122210415, 14.370900900900928, 26.574594594594597, 30.504504504504503, 11.730896396396398, 153.1044558071585, 0.864090429400791, 0.026034541543376477, 0.9500000000000001, 0.9512670565302148, 0.015393654524089306, 0.15865213794476282, 0.13962765957446807], {'avg_syllable_count': 1.2864864864864864, 'avg_wordlength': 6.848484848484849, 'avg_sentlengthbyword': 23.125, 'count_functionalwords': 0.42162162162162165, 'punctuation_count_ratio': 0.021253985122210415, 'fleschreadingease': 14.370900900900928, 'kincaid': 26.574594594594597, 'gunningfogindex': 30.504504504504503, 'dalechallindex': 11.730896396396398, 'yulek': 153.1044558071585, 'herdan': 0.864090429400791, 'maas': 0.026034541543376477, 'msttr': 0.9500000000000001, 'mattr': 0.9512670565302148, 'simpsond': 0.015393654524089306, 'sem_spread': 0.15865213794476282, 'pos_sem_spread': 0.13962765957446807})


In [ ]:
columns = vectors[0][1].keys()
columns

dict_keys(['avg_syllable_count', 'avg_wordlength', 'avg_sentlengthbyword', 'count_functionalwords', 'punctuation_count_ratio', 'fleschreadingease', 'kincaid', 'gunningfogindex', 'dalechallindex', 'yulek', 'herdan', 'maas', 'msttr', 'mattr', 'simpsond', 'sem_spread', 'pos_sem_spread'])

In [ ]:
dict2df={}
count=-1
for column in columns:
  count+=1
  temp = []
  for vector in vectors:
    temp.append(vector[0][count])
  dict2df[column]=temp

In [ ]:
print(len(dict2df['avg_syllable_count']))

2997


In [ ]:
vector_df = pd.DataFrame(dict2df)

In [ ]:
vector_df.head()

,avg_syllable_count,avg_wordlength,avg_sentlengthbyword,count_functionalwords,punctuation_count_ratio,fleschreadingease,kincaid,gunningfogindex,dalechallindex,yulek,herdan,maas,msttr,mattr,simpsond,sem_spread,pos_sem_spread
0,1.286486,6.848485,23.125000,0.421622,0.021254,14.370901,26.574595,30.504505,11.730896,153.104456,0.864090,0.026035,0.950000,0.951267,0.015394,0.158652,0.139628
1,1.296970,6.232558,23.571429,0.533333,0.018006,31.544545,22.523030,26.363636,11.436439,276.216713,0.766170,0.045796,0.926667,0.914349,0.027790,0.156127,0.118263
2,1.390909,5.956522,13.750000,0.381818,0.038532,25.507905,23.489234,25.803604,12.221642,183.339267,0.884998,0.024660,0.923810,0.936232,0.018509,0.160049,0.100446
3,1.198582,5.175824,20.142857,0.347518,0.026772,54.330256,17.523077,23.542191,11.963424,185.986159,0.876118,0.025217,0.918519,0.924590,0.018736,0.187371,0.090753
4,1.381503,5.651515,15.727273,0.260116,0.032511,39.154464,18.666786,22.300000,13.024786,135.904500,0.897966,0.019983,0.933333,0.931567,0.013673,0.172439,0.099719


In [ ]:
combined_df = pd.concat([df, vector_df], axis=1)

In [ ]:
combined_df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
combined_df.dropna(inplace=True)

In [ ]:
combined_df.shape

(2994, 20)

In [ ]:
combined_df.to_csv('style_extraction.csv')

##Modelling

In [ ]:
styledf = pd.read_csv('/content/drive/MyDrive/aiGenTxt/style_extraction.csv')